In [2]:
#Date Created: 2024-08-21
#Created by Z-bair
#
#Read Email and send to Telebot(AlerJSC)

In [ ]:
import imaplib
import email
from email.header import decode_header
from bs4 import BeautifulSoup
import asyncio
from telegram import Bot

def check_email(username, password):
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    mail.login(username, password)
    mail.select("inbox")

    # Mengambil email dari yang paling baru
    status, messages = mail.search(None, 'ALL')
    messages = messages[0].split()
    latest_msg_num = messages[-1]  # Ambil nomor pesan yang terbaru

    # Mengambil email terbaru
    status, msg_data = mail.fetch(latest_msg_num, '(RFC822)')
    msg = email.message_from_bytes(msg_data[0][1])

    # Mendekode subject
    subject, encoding = decode_header(msg["Subject"])[0]
    if isinstance(subject, bytes):
        subject = subject.decode(encoding)

    # Menyiapkan pesan
    message_content = f"Subject: {subject}\n\n"

    # Mendapatkan isi email
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            content_disposition = str(part.get("Content-Disposition"))

            if "attachment" not in content_disposition:
                try:
                    body = part.get_payload(decode=True).decode()
                    # Menghapus tag HTML dengan BeautifulSoup
                    soup = BeautifulSoup(body, 'html.parser')
                    text = soup.get_text()
                    message_content += text
                except:
                    pass
    else:
        body = msg.get_payload(decode=True).decode()
        # Menghapus tag HTML dengan BeautifulSoup
        soup = BeautifulSoup(body, 'html.parser')
        text = soup.get_text()
        message_content += text

    # Print subject dan isi email untuk debugging
    print(message_content)
    return message_content

    mail.logout()

# Fungsi untuk mengirim pesan melalui bot Telegram
async def send_message(token, chat_id, message):
    bot = Bot(token=token)
    async with bot:
        await bot.send_message(chat_id=chat_id, text=message)

# Fungsi untuk menjalankan coroutine di dalam event loop yang sudah berjalan
def run_asyncio_coroutine(coroutine):
    loop = asyncio.get_event_loop()
    if loop.is_running():
        task = loop.create_task(coroutine)
        return task
    else:
        loop.run_until_complete(coroutine)

# Implementasi di bot Telegram
username = "z00b41r@gmail.com"
password = "ftmkwzlcohietbpa"
TOKEN = "7367518171:AAHUUBLSuzbTwQ8vLiQJXpcJ2E0KsXCfhVg"  # Ganti dengan token bot Telegram Anda
CHAT_ID = "-1002168958382"  # Ganti dengan chat ID Anda
email_content = check_email(username, password)

if email_content:
    run_asyncio_coroutine(send_message(TOKEN, CHAT_ID, f"New Email:\n\n{email_content}"))